In [ ]:
'''
    File name: Retail Data Analysis.ipynb
    Author: Yue Liang
    Date last modified: 5/16/2021
    Python Version: 3.8
'''

In [1]:
import pandas as pd # for dataframes
import datetime as dt

In [2]:
data = pd.read_excel("Online_Retail.xlsx")

In [3]:
data

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


In [4]:
data= data[pd.notnull(data['InvoiceNo'])]
data= data[pd.notnull(data['StockCode'])]
data= data[pd.notnull(data['Description'])]
data= data[pd.notnull(data['Quantity'])]
data= data[pd.notnull(data['InvoiceDate'])]
data= data[pd.notnull(data['UnitPrice'])]
data= data[pd.notnull(data['CustomerID'])]
data= data[pd.notnull(data['Country'])]

In [5]:
data['CustomerID'] = data['CustomerID'].astype(int)
data = data[(data['Quantity']>0)]

In [6]:
data.describe()

,Quantity,UnitPrice,CustomerID
count,397924.000000,397924.000000,397924.000000
mean,13.021823,3.116174,15294.315171
std,180.420210,22.096788,1713.169877
min,1.000000,0.000000,12346.000000
25%,2.000000,1.250000,13969.000000
50%,6.000000,1.950000,15159.000000
75%,12.000000,3.750000,16795.000000
max,80995.000000,8142.750000,18287.000000


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 397924 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    397924 non-null  object        
 1   StockCode    397924 non-null  object        
 2   Description  397924 non-null  object        
 3   Quantity     397924 non-null  int64         
 4   InvoiceDate  397924 non-null  datetime64[ns]
 5   UnitPrice    397924 non-null  float64       
 6   CustomerID   397924 non-null  int32         
 7   Country      397924 non-null  object        
dtypes: datetime64[ns](1), float64(1), int32(1), int64(1), object(4)
memory usage: 25.8+ MB


In [8]:
data=data[['CustomerID','InvoiceDate','InvoiceNo','Quantity','UnitPrice']]

In [9]:
data['TotalPrice'] = data['Quantity'] * data['UnitPrice']

In [10]:
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'])
data['InvoiceDate'].min(),data['InvoiceDate'].max()

(Timestamp('2010-12-01 08:26:00'), Timestamp('2011-12-09 12:50:00'))

In [11]:
PRESENT = dt.datetime(2011,12,10)

In [12]:
data.head()

,CustomerID,InvoiceDate,InvoiceNo,Quantity,UnitPrice,TotalPrice
0,17850,2010-12-01 08:26:00,536365,6,2.55,15.30
1,17850,2010-12-01 08:26:00,536365,6,3.39,20.34
2,17850,2010-12-01 08:26:00,536365,8,2.75,22.00
3,17850,2010-12-01 08:26:00,536365,6,3.39,20.34
4,17850,2010-12-01 08:26:00,536365,6,3.39,20.34


In [13]:
rfm= data.groupby('CustomerID').agg({'InvoiceDate': lambda date: (PRESENT - date.max()).days,
                                        'InvoiceNo': lambda num: len(num),
                                        'TotalPrice': lambda price: price.sum()})

In [14]:
rfm.columns

Index(['InvoiceDate', 'InvoiceNo', 'TotalPrice'], dtype='object')

In [15]:
rfm.columns=['recency','frequency','monetary']

In [16]:
rfm['recency'] = rfm['recency'].astype(int)

In [17]:
rfm.head()

,recency,frequency,monetary
CustomerID,,,
12346,325,1,77183.60
12347,2,182,4310.00
12348,75,31,1797.24
12349,18,73,1757.55
12350,310,17,334.40


In [18]:
rfm['r_quartile'] = pd.qcut(rfm['recency'], 4, ['1','2','3','4'])
rfm['f_quartile'] = pd.qcut(rfm['frequency'], 4, ['4','3','2','1'])
rfm['m_quartile'] = pd.qcut(rfm['monetary'], 4, ['4','3','2','1'])

In [19]:
rfm.head()

,recency,frequency,monetary,r_quartile,f_quartile,m_quartile
CustomerID,,,,,,
12346,325,1,77183.60,4,4,1
12347,2,182,4310.00,1,1,1
12348,75,31,1797.24,3,3,1
12349,18,73,1757.55,2,2,1
12350,310,17,334.40,4,4,3


In [20]:
rfm['RFM_Segment_Concat'] = rfm.r_quartile.astype(str)+ rfm.f_quartile.astype(str) + rfm.m_quartile.astype(str)
rfm['RFM_Score'] = rfm.r_quartile.astype(int)+rfm.f_quartile.astype(int)+rfm.m_quartile.astype(int)

In [21]:
rfm

,recency,frequency,monetary,r_quartile,f_quartile,m_quartile,RFM_Segment_Concat,RFM_Score
CustomerID,,,,,,,,
12346,325,1,77183.60,4,4,1,441,9
12347,2,182,4310.00,1,1,1,111,3
12348,75,31,1797.24,3,3,1,331,7
12349,18,73,1757.55,2,2,1,221,5
12350,310,17,334.40,4,4,3,443,11
...,...,...,...,...,...,...,...,...
18280,277,10,180.60,4,4,4,444,12
18281,180,7,80.82,4,4,4,444,12
18282,7,12,178.05,1,4,4,144,9


In [22]:
rfm[rfm['RFM_Score']==3].sort_values('monetary', ascending=False).head()

,recency,frequency,monetary,r_quartile,f_quartile,m_quartile,RFM_Segment_Concat,RFM_Score
CustomerID,,,,,,,,
14646,1,2080,280206.02,1,1,1,111,3
18102,0,431,259657.30,1,1,1,111,3
17450,8,337,194550.79,1,1,1,111,3
14911,1,5677,143825.06,1,1,1,111,3
14156,9,1400,117379.63,1,1,1,111,3


Sort dataset by RFM_Score

In [23]:
rfm.sort_values('RFM_Score', ascending=True)

,recency,frequency,monetary,r_quartile,f_quartile,m_quartile,RFM_Segment_Concat,RFM_Score
CustomerID,,,,,,,,
16592,4,216,4113.68,1,1,1,111,3
14110,3,156,5683.15,1,1,1,111,3
14121,3,159,2780.15,1,1,1,111,3
14125,10,167,2740.43,1,1,1,111,3
14132,2,200,3586.03,1,1,1,111,3
...,...,...,...,...,...,...,...,...
12837,173,12,134.10,4,4,4,444,12
16498,161,15,100.97,4,4,4,444,12
17245,204,9,171.45,4,4,4,444,12
